# Mundipharma

## Beachten im nächsten Jahr
* Seitenenden werden nicht erkannt. Einige Adresse manuell einfügen!
* Totale wurden manuell berechnet
* Letzten drei Zeilen entfernt
* Columnen und Area mussten definiert werden

In [54]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [38]:
# Read pdf into DataFrame

columns = [21, 47.09, 233.45, 303.71, 348.85, 476.36, 512.02, 553.34, 596.85, 638.58, 680.28, 721.7, 763.5, 785.27]
area = [28.9559518348868, 12.820151963112876, 581.465980673998, 825.2729943703393]

df = tabula.read_pdf("Zuwendungen_an_medizinische_Fachkreise_2017.pdf", pages='all', lattice=True, columns=columns, guess=False, area=area)

## Add rows manually!

In [61]:
def add_rows_manually(df_export):
    #Add Missing rows EDIT!
    df_export = df_export.append({'name': 'Barbara Mühlheim', 
                                 'location': 'Bern',
                                 'country': 'Switzerland',
                                 'address': 'KODA-1',
                                 'registration_fees': 162,
                                 'travel_accommodation': 606,
                                 'total': 768,
                                 'type': 'hcp'
                                 }, ignore_index=True)

    df_export = df_export.append({'name': 'European League Against Pain EULAP', 
                                 'location': 'Zürich',
                                 'country': 'Switzerland',
                                 'address': 'Zürich',
                                 'donations_grants': 5000,
                                 'total': 5000,
                                 'type': 'hco'
                                 }, ignore_index=True)

    df_export = df_export.append({'name': 'Spitalverband Appenzell Ausserrhoden', 
                                 'location': 'Herisau',
                                 'country': 'Switzerland',
                                 'address': 'Herisau',
                                 'donations_grants': 1500,
                                 'total': 1500,
                                 'type': 'hco'
                                 }, ignore_index=True)
    return df_export

## Format Table

In [69]:
#Rename Columns
df_export = df.copy()
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total', 'empty1', 'empty2']

#Shift all where 'name' is empty
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Type by text
index_hco = df_export[df_export['name'].str.contains("INDIVIDUAL NAMED DISCLOSURE", na=False)].index[1]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove empty rows
df_export = df_export.dropna(subset=['location'], how='all')
df_export = df_export[~df_export.name.str.contains('Full Name')]
df_export = df_export[0: -3]

#Add Rows Manually
df_export = add_rows_manually(df_export)

#Remove empties
df_export.drop(columns=['empty1', 'empty2'], inplace=True)

#Cleanup Numbers and sum
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Add empty fields
df_export = add_plz(df_export)

#Remove Carination
df_export = remove_carination(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'mundipharma')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')